## 채용공고 scrapping

In [ ]:
# pip install selenium

In [1]:
from selenium import webdriver

In [2]:
browser = webdriver.Chrome('./chromedriver.exe')

In [3]:
browser.get('https://www.jobindexworld.com/jobpost/list')

In [4]:
html = browser.page_source

In [5]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(html, 'html.parser')

### 채용공고 list를 클릭해서 들어가기

In [ ]:
recruit_title = [] # 주제
company_name = [] # 회사명
company = []
detail_uri = [] # 내용 url

apply_start_date = [] # 모집기간
apply_end_date = []

task = [] # 직무개요
need_career = [] # 경력요건
team_env = [] # 처우
need_doc = [] # 제출서류

contact_people = [] # 담당컨설턴트
hashtag =[] # 해시태그

In [ ]:
import time

for i in range(5):
    # click해서 들어간 창
    browser.find_elements_by_css_selector('article.cpn-circle-conts-item')[i].click() # 클릭 페이지
    html_scraping = browser.page_source 
    soup_scraping = BeautifulSoup(html_scraping, 'html.parser')
    
    recruit = soup_scraping.select('article.circle-conts-view > h3.tit-h3') # 주제, 제목
    recruit_t = recruit[0].text.strip()
    recruit_title.append(recruit_t)
    
    uri = soup_scraping.select('meta[property="og:url"]') # 구직 회사 내용 화면
    uri_t = uri[0]['content']
    detail_uri.append(uri_t)
    
    # 
    start_date = soup_scraping.select('div.date')
    start_date_t = start_date[0].text.strip()[1:9] # 날짜, 시간 -> data time: https://docs.python.org/ko/3/library/datetime.html
    apply_start_date.append(start_date_t)
        
    tags_li = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div > ul > li')

    tags_li_data = []

    for i in range(len(tags_li)):
        tags_li_data.append(tags_li[i].text.strip().replace(' ','').replace('\n','').split(':'))

    for i in range(len(tags_li_data)):
        if tags_li_data[0][0] == '회사개요':
            company.append(tags_li_data[0][1])
            print(tags_li_data[0][1])
        elif tags_li_data[1][0] == '경력레벨':
            need_career.append(tags_li_data[1][1])
            print(tags_li_data[1][1])
        elif tags_li_data[2][0] == '급여수준':
            team_env.append(tags_li_data[2][1])
            print(tags_li_data[2][1])
        elif tags_li_data[3][0] == '교육수준':
            need_doc.append(tags_li_data[3][1])
            print(tags_li_data[3][1])
        elif tags_li_data[4][0] == '모집기간':
            apply_end_date.append(tags_li_data[4][1])
            print(tags_li_data[4][1])
            
    # 글자 수 대신 해당 row에 '찾는글자'가 있으면 으로 바꾸기
#     for i in range(len(tags)):
#         if tags_li[0].text.strip()[:4] == '회사개요':
#             task_t = tags_li[i].text.strip()[8:]
#             task.append(task_t)
#         elif tags_li[0].text.strip()[:4] == '경력레벨':
#             need_career_t = tags_li[i].text.strip()[-6:]
#             need_career.append(need_career_t)
#         elif tags_li[i].text.strip()[:4] == '급여수준':
#             team_env_t = tags_li[i].text.strip()[7:]
#             team_env.append(team_env_t)
#             # print(tags[i+1].text.strip())
#         elif tags_li[i].text.strip()[:4] == '교육수준':
#             need_doc_t = tags_li[i].text.strip()[8:]
#             need_doc.append(need_doc_t)
#         elif tags_li[i].text.strip()[:4] == '모집기간':
#             apply_end_date_t = tags_li[i].text.strip()[8:]
#             apply_end_date.append(apply_end_date_t)
            
    tags = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div')
    
#     for i in range(len(tags)):
#         if tags[i].text.strip()[2:7] == '직무 개요':
#             task_t = tags[i+1].text.strip()
#             task.append(task_t)
#             # print(tags[i+1].text.strip())
#         elif tags[i].text.strip()[2:7] == '경력 요건':
#             need_career_t = tags[i+1].text.strip()
#             need_career.append(need_career_t)
#             # print(tags[i+1].text.strip())
#         elif tags[i].text.strip()[2:4] == '처우':
#             team_env_t = tags[i+1].text.strip()
#             team_env.append(team_env_t)
#             # print(tags[i+1].text.strip())
#         elif tags[i].text.strip()[2:7] == '제출 기한':
#             apply_end_date_t = tags[i+1].text.strip()
#             apply_end_date.append(apply_end_date_t)
#             # print(tags[i+1].text.strip())
#         elif tags[i].text.strip()[2:12] == '제출 서류 및 기타':
#             need_doc_t = tags[i+1].text.strip()
#             need_doc.append(need_doc_t)
#             # print(tags[i+1].text.strip())        

    contact = soup_scraping.select('span.writer-info')
    contact_t = contact[0].text.strip()[0:3]
    contact_people.append(contact_t)
    
    hash_col = []
    hash = soup_scraping.select('div.keyword-wrap > a.keyword') # 해시태그
    for j in range(len(hash)):  
        hash_t = hash[j].text.strip()
        hash_col.append(hash_t)
    hashtag.append(hash_col)
    
    time.sleep(1)
    browser.back()


In [ ]:
recruit_title, detail_uri, contact_people, hashtag, apply_start_date

In [ ]:
apply_end_date, task, need_career, team_env, need_doc

In [7]:
browser.find_elements_by_css_selector('article.cpn-circle-conts-item')[7].click()

In [8]:
html_scraping = browser.page_source

In [9]:
soup_scraping = BeautifulSoup(html_scraping, 'html.parser')

In [ ]:
tags = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div')

In [10]:
company = []
apply_end_date = []
task = []
need_career = []
team_env = []
apply_end_date2 = []
need_doc = []

In [11]:
# 윗태그 2개 아래태그 5개 스크래핑 양식


tags_li = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div > ul > li')
tags_li_data = []

for i in range(len(tags_li)):
    tags_li_data.append(tags_li[i].text.strip().replace(' ','').replace('\n','').split(':'))

print(tags_li_data)
print('--------------')

for i in range(len(tags_li_data)):
    if tags_li_data[i][0] == '회사개요':
        company.append(tags_li_data[i][1])
    elif tags_li_data[i][0] == '모집기간':
        apply_end_date.append(tags_li_data[i][1])

print(company)
print(apply_end_date)

tags = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div')
tags = tags[1:len(tags)]
tags_data = []
for i in range(len(tags)):
    tags_data.append(tags[i].text.strip().replace('◈ ',''))
    
print(tags_data)
print('--------------')

for i in range(len(tags_data)):
    if tags_data[i] == '직무 개요':
        task.append(tags_data[i+1])

    elif tags_data[i] == '경력 요건':
        need_career.append(tags_data[i+1])

    elif tags_data[i] == '처우':
        team_env.append(tags_data[i+1])

    elif tags_data[i] == '제출 기한':
        apply_end_date2.append(tags_data[i+1])

    elif tags_data[i] == '제출 서류 및 기타':
        need_doc.append(tags_data[i+1])
        
print(task)
print(need_career)
print(team_env)
print(apply_end_date2)
print(need_doc)

[['회사개요', '국내모바일게임기업'], ['모집기간', '채용시까지']]
--------------
['국내모바일게임기업']
['채용시까지']
['직무 개요', '1. 언론홍보 전반', '', '경력 요건', '- 언론 홍보 관련 업무 경력 6년 이상- 홍보 업무 및 사회공헌 업무 등을 위한 전략적 기획력 보유- 각종 홍보 자료 작성 경험 및 스킬- 미디어 네트워크 및 원활한 커뮤니케이션 스킬 보유\xa0- 원활한 대인관계 및 팀워크 우수- 게임 및 게임 산업 전반에 대한 관심과 이해', '', '우대 사항', '- 기업 홍보실/홍보팀 경력- 게임 또는 IT 업계 경험자\xa0- 게임 및 IT관련 미디어 네트워크- 영어 능통자', '처우', '- 협의', '', '제출 기한', '- ASAP', '', '제출 서류 및 기타', '- 국문 이력서 및 자기소개서', '', '담당 컨설턴트', '- 이재환 / 02-2016-6694 / jhlee@vpeople.co.kr']
--------------
['1. 언론홍보 전반']
['- 언론 홍보 관련 업무 경력 6년 이상- 홍보 업무 및 사회공헌 업무 등을 위한 전략적 기획력 보유- 각종 홍보 자료 작성 경험 및 스킬- 미디어 네트워크 및 원활한 커뮤니케이션 스킬 보유\xa0- 원활한 대인관계 및 팀워크 우수- 게임 및 게임 산업 전반에 대한 관심과 이해']
['- 협의']
['- ASAP']
['- 국문 이력서 및 자기소개서']


In [12]:
company, apply_end_date, task, need_career, apply_end_date, need_doc

(['국내모바일게임기업'],
 ['채용시까지'],
 ['1. 언론홍보 전반'],
 ['- 언론 홍보 관련 업무 경력 6년 이상- 홍보 업무 및 사회공헌 업무 등을 위한 전략적 기획력 보유- 각종 홍보 자료 작성 경험 및 스킬- 미디어 네트워크 및 원활한 커뮤니케이션 스킬 보유\xa0- 원활한 대인관계 및 팀워크 우수- 게임 및 게임 산업 전반에 대한 관심과 이해'],
 ['채용시까지'],
 ['- 국문 이력서 및 자기소개서'])

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(tags_li_data, columns=['tags_li'])

In [ ]:
df['tags_li'][0].replace('[^가-힣ㄱ-ㅎㅠ ]','')

In [ ]:
import time

for i in range(10):
    # click해서 들어간 창
    browser.find_elements_by_css_selector('article.cpn-circle-conts-item')[i].click() # 클릭 페이지
    html_scraping = browser.page_source 
    soup_scraping = BeautifulSoup(html_scraping, 'html.parser')
        
    tags_li = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div > ul > li')
    tags_li_data = []

    for i in range(len(tags_li)):
        tags_li_data.append(tags_li[i].text.strip().replace(' ','').replace('\n','').split(':'))

    print(tags_li_data)
    
    print('-----------------')
    task.append(tags_li_data[0][1])
    need_career.append(tags_li_data[1][1])
    team_env.append(tags_li_data[2][1])
    need_doc.append(tags_li_data[3][1])
    apply_end_date.append(tags_li_data[4][1])
          
    tags = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div')
    
    task.append(tags_li_data[0][1])
    
    apply_end_date.append(tags_li_data[4][1])
    
    for i in range(len(tags)):
        if tags[i].text.strip()[2:7] == '직무 개요':
            task_t = tags[i+1].text.strip()
            task.append(task_t)
            # print(tags[i+1].text.strip())
        elif tags[i].text.strip()[2:7] == '경력 요건':
            need_career_t = tags[i+1].text.strip()
            need_career.append(need_career_t)
            # print(tags[i+1].text.strip())
        elif tags[i].text.strip()[2:4] == '처우':
            team_env_t = tags[i+1].text.strip()
            team_env.append(team_env_t)
            # print(tags[i+1].text.strip())
        elif tags[i].text.strip()[2:7] == '제출 기한':
            apply_end_date_t = tags[i+1].text.strip()
            apply_end_date.append(apply_end_date_t)
            # print(tags[i+1].text.strip())
        elif tags[i].text.strip()[2:12] == '제출 서류 및 기타':
            need_doc_t = tags[i+1].text.strip()
            need_doc.append(need_doc_t)
            # print(tags[i+1].text.strip())        
   
    time.sleep(1)
    browser.back()

In [ ]:
apply_end_date, task, need_career, team_env, need_doc

In [ ]:
start_date = soup_scraping.select('div.date')
start_date_t = start_date[0].text.strip()
start_date[0].replace('[^가-힣ㄱ-ㅎㅠ ]','')

### tag를 활용하여 select

In [ ]:
recruit_title = soup.select('article.circle-conts-view > h3.tit-h3') # 주제, 제목
company_name = soup.select('div.jobpost-summary-wrapper > ul > li') # 회사명 -> 회사개요
detail_uri = soup.select('meta[property="og:url"]') # 구직 회사 내용 화면
apply_start_date = soup.select('') # 모집기한
apply_end_date = soup.select('')
task = soup.select('') # 직무개요 div.circle-conts-view-main-conts-box-question > div [3,4]
need_career = soup.select('') # 경력요건  div.circle-conts-view-main-conts-box-question > div [7]
team_env = soup.select('') # 처우  div.circle-conts-view-main-conts-box-question > div [10]
need_doc = soup.select('') # 제출서류  div.circle-conts-view-main-conts-box-question > div []
contact_people = soup.select('') # 담당컨설턴트  div.circle-conts-view-main-conts-box-question > div
hashtag = soup.select('div.keyword-wrap') # 해시태그

In [ ]:
recruit_title = soup.select('article.circle-conts-view > h3.tit-h3') # 주제, 제목

In [ ]:
len(recruit_title), type(recruit_title), print(recruit_title)

In [ ]:
company_name = soup.select('div.jobpost-summary-wrapper > ul > li') # 회사명

In [ ]:
print(company_name)

In [ ]:
detail_uri = soup.select('meta[property="og:url"]') # 구직 회사 내용 화면

In [ ]:
print(detail_uri)

In [ ]:
detail_uri[0].text

In [ ]:
hashtag = soup.select('div.keyword-wrap') # 해시태그

In [ ]:
len(hashtag), type(hashtag)

In [ ]:
print(hashtag)

In [ ]:
hashtag[0].text

In [ ]:
contents = list()
for hash in hashtag:
    h_tag = hash.select('div.keyword-wrap')
    contents.append(h_tag)

contents

### SCRAPING SITE TABLE에 넣기

In [ ]:
brawser = webdriver.Chrome('./chromedriver.exe')

In [ ]:
brawser.get('https://github.com/login')

In [ ]:
brawser.find_elements_by_css_selector('input#login_field')[0].send_keys('superinssa@gmail.com')

In [ ]:
brawser.find_elements_by_css_selector('input#password')[0].send_keys('ghlehd123#')

In [ ]:
brawser.find_elements_by_css_selector('input[type="submit"]')[0].click()